In [1]:
import pandas as pd
import numpy as np

In [5]:
f = '../data/'

In [6]:
# Identify acute myocardial infarction (AMI) diagnoses
df_diag = pd.read_csv(f + 'diagnoses_icd.csv')
df_icd = pd.read_csv(f + 'd_icd_diagnoses.csv')

# Merge to get the text description (just for us to know about)
df_diag = df_diag.merge(df_icd, on=['icd_code', 'icd_version'], how='left')

# Select ICD-9 = 410.* or ICD-10 = I21.*
df_ami = df_diag[
    ((df_diag['icd_version'] == 9) & (df_diag['icd_code'].str.startswith('410'))) |
    ((df_diag['icd_version'] == 10) & (df_diag['icd_code'].str.startswith('I21')))
]

In [9]:
df_ami['subject_id'].nunique()

13147

In [11]:
df_adm = pd.read_csv(f + 'admissions.csv')
df_pat = pd.read_csv(f + 'patients.csv')

# Merge with admissions and patients so each AMI diagnosis is linked to demographics and hospital stay info
df_merged = df_ami.merge(
    df_adm, 
    on=['subject_id','hadm_id'], 
    how='left', 
    validate='m:1'
)
df_merged = df_merged.merge(
    df_pat, 
    on='subject_id', 
    how='left', 
    validate='m:1'
)

# Restricting to adults
df_merged = df_merged[df_merged['anchor_age'] >= 18] # to account for anchor_age being shifted

In [12]:
df_merged['subject_id'].nunique()

13147

In [13]:
df_icu = pd.read_csv(f + 'icustays.csv')

# Merge ICU info
df_icu_merged = df_merged.merge(
    df_icu, 
    on=['hadm_id', 'subject_id'], 
    how='inner', 
    validate='m:m'
)

# Keep the first ICU stay per patient
df_icu_merged = (
    df_icu_merged.sort_values(['subject_id', 'intime'])
    .groupby('subject_id')
    .first()
    .reset_index()
)

In [14]:
df_icu_merged['subject_id'].nunique()

7630

In [155]:
# To calculate how long patient was in the ICU
df_icu_merged['intime'] = pd.to_datetime(df_icu_merged['intime'], errors='coerce')
df_icu_merged['outtime'] = pd.to_datetime(df_icu_merged['outtime'], errors='coerce')

In [156]:
# Merging with lab values

df_lab = pd.read_csv(
    f + 'labevents.csv',
    usecols=['subject_id', 'hadm_id', 'itemid', 'charttime', 'valuenum', 'valueuom']
)
df_lab['charttime'] = pd.to_datetime(df_lab['charttime'], errors='coerce')

# Reading d_labitems (not to be used in model, for our own readability)
df_dlab = pd.read_csv(f + 'd_labitems.csv', usecols=['itemid', 'label'])
df_dlab['label'] = df_dlab['label'].astype(str).str.strip()

# Keep only labs from admissions in the ICU cohort
hadm_keep = set(df_icu_merged['hadm_id'].unique())
df_lab = df_lab[df_lab['hadm_id'].isin(hadm_keep)]

# Attach test names (not to be used in model, for our own readability)
df_lab = df_lab.merge(df_dlab, on='itemid', how='left')
df_lab['label'] = df_lab['label'].astype(str).str.strip()

In [216]:
df_lab.to_pickle(f + 'df_lab.pkl')

In [219]:
# Join ICU times and restrict to first 24 hours after ICU admission
labs = df_lab.merge(
    df_icu_merged[['subject_id', 'hadm_id', 'stay_id', 'intime']],
    on=['subject_id', 'hadm_id'],
    how='inner'
)

mask_24h = (
    (labs['charttime'] >= labs['intime']) & 
    (labs['charttime'] < labs['intime'] + pd.Timedelta(hours=24))
)
labs_24h = labs.loc[mask_24h].copy()

In [220]:
# Converting #/uL to K/uL

mask = labs_24h['valueuom'] == '#/uL'
labs_24h.loc[mask, 'valuenum'] = labs_24h.loc[mask, 'valuenum'] / 1000
labs_24h.loc[mask, 'valueuom'] = 'K/uL'

In [221]:
# Calculate SIRI
labs = labs_24h.copy()
labs["valuenum"] = pd.to_numeric(labs["valuenum"], errors="coerce")

# Keep only the three components needed to calcualte SIRI
need = {
    "Absolute Lymphocyte Count",
    "Absolute Monocyte Count",
    "Absolute Neutrophil Count",
}
src = labs[labs["label"].isin(need)].copy()

# Pivot by draw time so all three are on one row per (patient, encounter, time)
panel = (
    src.pivot_table(
        index=["subject_id","hadm_id","stay_id","intime","charttime"],
        columns="label",
        values="valuenum",
        aggfunc="first"
    )
    .reset_index()
)

# Require all three to be present and avoid division by zero
panel = panel.dropna(subset=list(need))
panel = panel[panel["Absolute Lymphocyte Count"] > 0]

# Compute SIRI = ANC * AMC / ALC
panel["valuenum"] = (
    panel["Absolute Neutrophil Count"] * panel["Absolute Monocyte Count"] / panel["Absolute Lymphocyte Count"]
)

# Build rows that look like labs_24h and append
siri_rows = panel[["subject_id","hadm_id","stay_id","intime","charttime","valuenum"]].copy()
siri_rows["itemid"] = -1000            # custom code for SIRI (choose any unused ID)
siri_rows["valueuom"] = "unitless"
siri_rows["label"] = "SIRI"

# Order columns to match labs_24h
siri_rows = siri_rows[labs_24h.columns]

# Append to original table
labs_24h_with_siri = pd.concat([labs_24h, siri_rows], ignore_index=True)
labs_24h = labs_24h_with_siri.copy()

In [222]:
# Some data exploration
# Ignore cell and move on tbh
mask_24count = labs_24h['label'].value_counts()[lambda x: x > 1000] #only keeping labs with greater than 1000 instances

common_labels = mask_24count.index  # extract label names

labs_common = labs_24h[labs_24h['label'].isin(common_labels)]
# Drop nans in value_num
labs_common = labs_common[labs_common['valuenum'].notna()]

# Filter the DataFrame
df_lab_filtered = labs_common[labs_common['label'].isin(common_labels)]

dupes = df_lab_filtered.duplicated(subset=['subject_id', 'hadm_id', 'itemid', 'label'], keep=False)
df_dupes = df_lab_filtered[dupes].sort_values(['subject_id', 'hadm_id', 'itemid'])
df_dupes_grouped = df_dupes.groupby(['subject_id','hadm_id','itemid']).size().sort_values(ascending=False)
df_dupes_grouped
df_dupes['label'].unique()


array(['Creatine Kinase, MB Isoenzyme', 'Troponin T', 'INR(PT)', 'PT',
       'PTT', 'Anion Gap', 'Bicarbonate', 'Calcium, Total', 'Chloride',
       'CK-MB Index', 'Creatine Kinase (CK)', 'Creatinine', 'Glucose',
       'Magnesium', 'Phosphate', 'Potassium', 'Sodium', 'Urea Nitrogen',
       'Hematocrit', 'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Platelet Count',
       'RDW', 'Red Blood Cells', 'White Blood Cells', 'Lactate', 'RDW-SD',
       'Base Excess', 'Calculated Total CO2', 'Chloride, Whole Blood',
       'Free Calcium', 'Hematocrit, Calculated', 'pCO2', 'pH', 'pO2',
       'Potassium, Whole Blood', 'Sodium, Whole Blood', 'H', 'I', 'L',
       'Oxygen Saturation', 'Temperature', 'Fibrinogen, Functional',
       'Oxygen', 'PEEP', 'Tidal Volume', 'Bilirubin, Total',
       'Lactate Dehydrogenase (LD)', 'SIRI', 'Absolute Lymphocyte Count',
       'Basophils', 'Eosinophils', 'Lymphocytes', 'Monocytes',
       'Neutrophils', 'Absolute Basophil Count',
       'Absolute Eosinophil Count',

In [223]:
worst_direction = {
    "max": [
        "Creatine Kinase, MB Isoenzyme",
        "Troponin T",
        "INR(PT)",
        "PT",
        "PTT",
        "Anion Gap",
        "CK-MB Index",
        "Creatine Kinase (CK)",
        "Creatinine",
        "Urea Nitrogen",
        "RDW",
        "Lactate",
        "RDW-SD",
        "H",
        "I",
        "L",
        "Temperature",
        "Bilirubin, Total",
        "Lactate Dehydrogenase (LD)",
        "Basophils",
        "Eosinophils",
        "Lymphocytes",
        "Monocytes",
        "Absolute Basophil Count",
        "Absolute Eosinophil Count",
        "Absolute Monocyte Count",
        "Absolute Neutrophil Count",
        "Immature Granulocytes",
        "Alanine Aminotransferase (ALT)",
        "Alkaline Phosphatase",
        "Asparate Aminotransferase (AST)",
        "Cholesterol Ratio (Total/HDL)",
        "Cholesterol, Total",
        "Triglycerides",
        "SIRI"
    ],

    "min": [
        "Bicarbonate",
        "Hematocrit",
        "Hemoglobin",
        "Platelet Count",
        "Red Blood Cells",
        "Base Excess",
        "Calculated Total CO2",
        "Hematocrit, Calculated",
        "Oxygen Saturation",
        "pO2",
        "Oxygen",
        "Fibrinogen, Functional",
        "Absolute Lymphocyte Count",
        "Albumin",
        "Cholesterol, HDL",
        "Protein",
        "Specific Gravity",
        "Ketone"
    ],

    "range": [
        "Calcium, Total",
        "Chloride",
        "Glucose",
        "Magnesium",
        "Phosphate",
        "Potassium",
        "Sodium",
        "MCH",
        "MCHC",
        "MCV",
        "White Blood Cells",
        "pCO2",
        "pH",
        "Free Calcium",
        "Chloride, Whole Blood",
        "Potassium, Whole Blood",
        "Sodium, Whole Blood",
        "Temperature",
        "RBC",
        "WBC",
        "Urobilinogen",
        "% Hemoglobin A1c",
        "eAG"
    ],

    "binary": [
        "PEEP",
        "Tidal Volume",
        "Specimen Type"
    ],
}


In [224]:
# Only taking labs with max and min as the worst values, and then only taking worst value in first 24 hours of ICU
labs_24h = labs_24h.dropna(subset=["valuenum"])

max_labs = worst_direction["max"]
df_max = labs_24h[labs_24h["label"].isin(max_labs)].copy()
df_max_one = (
    # Note that I use "label" instead of "itemid" because many equivolent tests (like Hb) can have multiple itemid 
    df_max.sort_values(["subject_id", "label", "valuenum", "charttime"], ascending=[True, True, False, True])
          .groupby(["subject_id", "label"], as_index=False)
          .head(1)
)

min_labs = worst_direction["min"]
df_min = labs_24h[labs_24h["label"].isin(min_labs)].copy()

df_min_one = (
    df_min.sort_values(["subject_id", "label", "valuenum", "charttime"], ascending=[True, True, True, True]) #breaking ties with earliest charttime
          .groupby(["subject_id", "label"], as_index=False)
          .head(1)
)

In [225]:
# merge the two DataFrames
df_extremes = pd.concat([df_min_one, df_max_one], ignore_index=True)

# sort by subject_id (and itemid, charttime)
df_extremes = df_extremes.sort_values(["subject_id", "itemid", "charttime"])

# reset the index
df_extremes = df_extremes.reset_index(drop=True)

In [226]:
# Checking for duplicate rows
dup_mask = df_extremes.duplicated(subset=["subject_id", "hadm_id", "label"], keep=False)
df_duplicates = df_extremes[dup_mask]

print(f"Number of duplicate rows: {df_duplicates.shape[0]}")

Number of duplicate rows: 0


In [227]:
labs_pivot = (
    df_extremes.pivot_table(
        index=['subject_id', 'hadm_id'],   # each patient/admission becomes one row
        columns='label',                   # each lab name becomes a column
        values='valuenum'                # use the lab value (we don't have units anymore)
    )
    .reset_index()
)

In [228]:
df_final = df_icu_merged.merge(labs_pivot, on=['subject_id', 'hadm_id'], how='left')

print("Final cohort size:", len(df_final))
df_final.head(20)

Final cohort size: 7630


,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title,admittime,dischtime,deathtime,admission_type,...,RDW,RDW-SD,Red Blood Cells,SIRI,Specific Gravity,Temperature,Triglycerides,Troponin T,Urea Nitrogen,pO2
0,10000980,26913865,1,41071,9,"Subendocardial infarction, initial episode of ...",2189-06-27 07:38:00,2189-07-03 03:00:00,None,EW EMER.,...,13.2,NaN,3.28,NaN,NaN,NaN,NaN,0.15,38.0,NaN
1,10002155,23822395,1,41011,9,Acute myocardial infarction of other anterior ...,2129-08-04 12:44:00,2129-08-18 16:53:00,None,EW EMER.,...,15.2,NaN,4.17,NaN,NaN,NaN,NaN,3.99,19.0,76.0
2,10002495,24982426,1,I214,10,Non-ST elevation (NSTEMI) myocardial infarction,2141-05-22 20:17:00,2141-05-29 17:41:00,None,URGENT,...,12.5,42.5,4.13,67.284455,1.032,NaN,90.0,3.75,36.0,NaN
3,10002667,23197839,1,I214,10,Non-ST elevation (NSTEMI) myocardial infarction,2187-02-23 16:01:00,2187-02-28 16:00:00,None,OBSERVATION ADMIT,...,13.8,39.7,3.61,NaN,1.033,NaN,NaN,NaN,10.0,140.0
4,10005593,26835370,1,I214,10,Non-ST elevation (NSTEMI) myocardial infarction,2125-06-23 18:37:00,2125-07-03 20:14:00,None,OBSERVATION ADMIT,...,13.7,46.1,2.42,0.778813,NaN,NaN,NaN,NaN,49.0,123.0
5,10005817,20626031,1,41071,9,"Subendocardial infarction, initial episode of ...",2132-12-12 01:43:00,2132-12-20 15:04:00,None,URGENT,...,14.8,NaN,2.30,NaN,NaN,NaN,NaN,NaN,16.0,79.0
6,10006053,22942076,3,41071,9,"Subendocardial infarction, initial episode of ...",2111-11-13 23:39:00,2111-11-15 17:20:00,2111-11-15 17:20:00,URGENT,...,15.4,NaN,2.72,NaN,1.022,37.1,NaN,NaN,18.0,50.0
7,10007058,22954658,1,I214,10,Non-ST elevation (NSTEMI) myocardial infarction,2167-11-07 19:05:00,2167-11-11 14:23:00,None,EW EMER.,...,13.0,41.0,4.32,NaN,NaN,NaN,NaN,1.82,11.0,NaN
8,10009686,29681222,1,41021,9,Acute myocardial infarction of inferolateral w...,2164-04-30 00:00:00,2164-05-03 18:45:00,None,URGENT,...,12.7,NaN,3.67,NaN,1.003,NaN,NaN,1.47,7.0,NaN
9,10010058,26359957,1,I2109,10,ST elevation (STEMI) myocardial infarction inv...,2147-11-18 02:34:00,2147-11-19 04:00:00,2147-11-19 04:00:00,EW EMER.,...,14.8,53.0,3.22,NaN,1.026,39.0,NaN,3.82,79.0,31.0


In [229]:
# Dropping empty columns (just in case)
df_final = df_final.dropna(axis=1, how='all')
df_final.columns

Index(['subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version',
       'long_title', 'admittime', 'dischtime', 'deathtime', 'admission_type',
       'admit_provider_id', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'race', 'edregtime',
       'edouttime', 'hospital_expire_flag', 'gender', 'anchor_age',
       'anchor_year', 'anchor_year_group', 'dod', 'stay_id', 'first_careunit',
       'last_careunit', 'intime', 'outtime', 'los', 'Absolute Basophil Count',
       'Absolute Eosinophil Count', 'Absolute Lymphocyte Count',
       'Absolute Monocyte Count', 'Absolute Neutrophil Count',
       'Alanine Aminotransferase (ALT)', 'Albumin', 'Alkaline Phosphatase',
       'Anion Gap', 'Asparate Aminotransferase (AST)', 'Base Excess',
       'Basophils', 'Bicarbonate', 'Bilirubin, Total', 'CK-MB Index',
       'Calculated Total CO2', 'Cholesterol Ratio (Total/HDL)',
       'Cholesterol, HDL', 'Cholesterol, Total', 'Creatine Kinase (CK)',
  

In [230]:
df_final.to_csv(f + 'final_cohort.csv', index=False)